In [ ]:
import numpy as np
import pandas as pd
import json
from elasticsearch import Elasticsearch

es = Elasticsearch(['https://localhost:9201'], basic_auth=('elastic', 'wfRM-cEYuJW0Cxo0nF_z'), verify_certs=False, ssl_show_warn=False)

# Fact Checking: retrieve related tables #

### optional: you can choose to retrieve tables for all textual claims in tabfact ###

In [ ]:
from nltk.corpus import stopwords
from tqdm import tqdm
import os

def load_all_the_data_tabfact():
    all_data = {}
    for fnm in [os.path.join(dataset_path, "collected_data/r1_training_all.json"), os.path.join(dataset_path, "collected_data/r2_training_all.json")]:
        infos = eval(open(fnm).read())
        all_data.update(infos)
    return all_data



def get_data_tabfact(which):
    # query_text : table_id
    all_querys = {}
    if which == 'test':
        fnm = os.path.join(dataset_path, 'test_id.json')
    table_ids = eval(open(fnm).read())
    for tab_id in tqdm(table_ids):
        tab_data = all_data[tab_id]
        qs, labels, caption = tab_data
        for i, query in enumerate(qs):
            query_text = caption + '. ' + query
            # query_text = query
            all_querys[query_text] = [tab_id.split('.')[0], labels[i]]

    return all_querys


dataset_path = './data/tabfact'
all_data = load_all_the_data_tabfact()
test_claims = get_data_tabfact('test')
print(len(test_claims))

In [ ]:
# 打开test_claim.json
test_claims = json.load(open('./data/tabfact/test_claims.json'))
print(len(test_claims))

In [ ]:
return_size = 50
recall = 0
recall_by_es = 0
retrieve_results = {}

for query_text, values in test_claims.items():
    tab_id = values[0]
    # retrieved by elasticsearch
    # Search for documents using n-grams
    index_name = 'table_index'
    query = {
        "query": {
            "match": {
                "table": query_text
            }
        }
    }

    final_results = es.search(index=index_name, body=query, size=return_size)
    retrieve_doc_2 = [ hit['_source']['table id'] for hit in final_results['hits']['hits'] ]
    retrieve_results[query_text] = retrieve_doc_2
    tab_id = tab_id[:tab_id.index('.')]

    if tab_id in retrieve_doc_2:
        recall_by_es += 1

print("recall_rate")
print(recall_by_es / len(test_claims))

In [ ]:
return_size = 5
recall_top5 = 0
retrieve_query = {}
for query_text, values in test_queries.items():
    tab_id = values[0]
    if query_text in retrieve_results:
        if tab_id in retrieve_results[query_text][:return_size]:
            recall_top5 += 1
        retrieve_query[query_text] = [values[0], values[1], retrieve_results[query_text][:return_size]]

print(recall_top5 / len(test_claims))

with open('retrieve_tables.json', 'w') as f_index:
    json.dump(retrieve_query, f_index)

# Missing Value Imputation: retrieve related tuples and text #

### optional: you can choose to retrieve tuples and text for all test tuples ###

In [ ]:
es = Elasticsearch(['https://localhost:9201'], basic_auth=('elastic', 'wfRM-cEYuJW0Cxo0nF_z'), verify_certs=False, ssl_show_warn=False)

return_size = 3
retrieved_tuples = {}
with open('./data/missing_value_imputation/test.jsonl', 'r') as f:
    for tuple_ in f:
        tuple_ = json.loads(tuple_)
        caption = tuple_['caption']
        pgTitle = tuple_['pgTitle']
        secTitle = tuple_['secTitle']
        headers = tuple_['attributes']
        tuple_id = tuple_['tuple id']
        values = tuple_['values']
        tuple_string = ''

        if pgTitle != '':
            tuple_string = 'pgTitle: ' + pgTitle + ' |'
        if secTitle != '':
            tuple_string += ' secTitle: ' + secTitle + ' |' 
        if caption != '':
            tuple_string += ' caption: ' + caption + ' |'


        mask_entity = tuple_['mask_entity'] 
        mask_column = values.index(mask_entity)

        for i in range(len(headers)):
            if values[i] == '':
                continue
            if i == mask_column:
                tuple_string += ' attribute ' + headers[i] + ' value <mask>'
            else:
                tuple_string += ' attribute ' + headers[i] + ' value ' + values[i] 

        retrieved_tuples[tuple_string] = {'retrieved_text': [], 'retrieved_tuples': []}

        query = {
            "query": {
                "match": {
                    "tuple": tuple_string
                }
            }
        }

        tuple_results = es.search(index='tuple_index', body=query, size=return_size)

        query = {
            "query": {
                "match": {
                    "text": tuple_string
                }
            }
        }
        text_results = es.search(index='text_index', body=query, size=return_size)

        for result in tuple_results['hits']['hits']:
            retrieved_tuples[tuple_string]['retrieved_tuples'].append(result['_source']['tuple'])
        for result in text_results['hits']['hits']:
            retrieved_tuples[tuple_string]['retrieved_text'].append(result['_source']['text'])



In [ ]:
es = Elasticsearch(['https://localhost:9201'], basic_auth=('elastic', 'wfRM-cEYuJW0Cxo0nF_z'), verify_certs=False, ssl_show_warn=False)

return_size = 3
retrieved_evidence = {}
with open('selected_tuples.txt', 'r') as f:
    lines = f.readlines()
    count = 1
    while count < len(lines):
        print("------------------------------")
        query_text = lines[count].strip()
        ground_truth = lines[count+1].strip()
        ans = lines[count+2].strip()
        query_text = query_text.replace('_',' ')
        print(query_text)
        print("ground_truth: " + ground_truth)

        retrieved_evidence[query_text] = {'retrieved_text': [], 'retrieved_tuples': []}

        query = {
            "query": {
                "match": {
                    "tuple": query_text
                }
            }
        }

        tuple_results = es.search(index='tuple_index', body=query, size=return_size)

        query = {
            "query": {
                "match": {
                    "text": tuple_string
                }
            }
        }
        text_results = es.search(index='text_index', body=query, size=return_size)

        for result in tuple_results['hits']['hits']:
            retrieved_evidence[tuple_string]['retrieved_tuples'].append(result['_source']['tuple'])
        for result in text_results['hits']['hits']:
            retrieved_evidence[tuple_string]['retrieved_text'].append(result['_source']['text'])


# 保存retrieved tuples 到json文件中
with open('./data/missing_value_imputation/retrieved_evidences.json', 'w') as f:
    json.dump(retrieved_evidence, f)